## Mine top 10 association rules worth recommending for Groceries dataset

In [1]:
import pandas as pd
df = pd.read_csv(r"C:\Users\User\Downloads\archive (1)\Groceries_dataset.csv")
df.head ()

,Member_number,Date,itemDescription
0,1808,21-07-2015,tropical fruit
1,2552,05-01-2015,whole milk
2,2300,19-09-2015,pip fruit
3,1187,12-12-2015,other vegetables
4,3037,01-02-2015,whole milk


In [2]:
from apyori import apriori

# Group transactions as lists of products
transactions = df.groupby(['Member_number', 'Date'])['itemDescription'].apply(list).values.tolist()

# Run the Apriori algorithm
rules = list(apriori(transactions, min_support=0.001, min_confidence=0.1, min_lift=1, min_length=2))

# Filter unique itemsets (remove duplicates)
seen_itemsets = set()
unique_rules = []
for rule in rules:
    itemset = frozenset(rule.items)
    if itemset not in seen_itemsets:
        seen_itemsets.add(itemset)
        unique_rules.append(rule)

# Sort by lift of the first ordered statistic and select top 10
sorted_unique_rules = sorted(
    unique_rules,
    key=lambda r: r.ordered_statistics[0].lift if r.ordered_statistics else 0,
    reverse=True
)[:10]

# Function to format rules as a table
def print_rules(rules):
    data = []
    for rule in rules:
        for os in rule.ordered_statistics:
            if os.items_base and os.items_add:
                data.append({
                    'Antecedent': list(os.items_base),
                    'Consequent': list(os.items_add),
                    'Support': rule.support,
                    'Confidence': os.confidence,
                    'Lift': os.lift
                })
                break  
    return pd.DataFrame(data)

print_rules(sorted_unique_rules)

,Antecedent,Consequent,Support,Confidence,Lift
0,"[whole milk, sausage]",[yogurt],0.001470,0.164179,1.911760
1,[flour],[tropical fruit],0.001069,0.109589,1.617141
2,[processed cheese],[root vegetables],0.001069,0.105263,1.513019
3,[soft cheese],[yogurt],0.001270,0.126667,1.474952
4,[detergent],[yogurt],0.001069,0.124031,1.444261
5,[chewing gum],[yogurt],0.001403,0.116667,1.358508
6,"[rolls/buns, sausage]",[whole milk],0.001136,0.212500,1.345594
7,[processed cheese],[rolls/buns],0.001470,0.144737,1.315734
8,[packaged fruit/vegetables],[rolls/buns],0.001203,0.141732,1.288421
9,[seasonal products],[rolls/buns],0.001002,0.141509,1.286395


In [3]:
len(rules)

33